### Прогнозирование диабета (очистка данных)

In [15]:
import pandas as pd
import numpy as np

In [3]:
diabetes = pd.read_csv('../data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [4]:
diabetes.shape

(778, 10)

In [7]:
# найдем полные дубли строк
diabetes.drop_duplicates(subset=diabetes.columns, inplace=True)
diabetes.shape

(768, 10)

In [11]:
# удалим неиформативные признаки
low_information_cols = [] 

for col in diabetes.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes[col].nunique() / diabetes[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
diabetes.drop(low_information_cols, axis=1, inplace=True)

Gender: 100.0% одинаковых значений


In [14]:
# найдем пропуски данных
cols_null_percent = diabetes.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

Series([], dtype: float64)

Ничего не нашли. Но пропуски есть, просто пропущенные значения = 0

In [21]:
diabetes.Glucose = diabetes.Glucose.apply(lambda x: np.nan if x==0 else x)
diabetes.BloodPressure = diabetes.BloodPressure.apply(lambda x: np.nan if x==0 else x)
diabetes.SkinThickness = diabetes.SkinThickness.apply(lambda x: np.nan if x==0 else x)
diabetes.Insulin = diabetes.Insulin.apply(lambda x: np.nan if x==0 else x)
diabetes.BMI = diabetes.BMI.apply(lambda x: np.nan if x==0 else x)

In [22]:
# найдем пропуски данных
cols_null_percent = diabetes.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

Insulin          48.697917
SkinThickness    29.557292
BloodPressure     4.557292
BMI               1.432292
Glucose           0.651042
dtype: float64

In [25]:
# удалим из данных признаки, где число пропусков составляет более 30 %
diabetes.dropna(axis=1, thresh=diabetes.shape[0]*0.7, inplace=True)
diabetes.shape

(768, 8)

In [27]:
# удалим строки с количеством пропусков > 2
diabetes.dropna(axis=0, thresh=diabetes.shape[1]-2, inplace=True)
diabetes.shape

(761, 8)

In [28]:
cols_null_percent = diabetes.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
display(cols_with_null)

SkinThickness    28.909330
BloodPressure     3.679369
Glucose           0.657030
BMI               0.525624
dtype: float64

In [30]:
# в оставшихся записях заменим пропуски на медиану
diabetes.fillna({
    'SkinThickness': diabetes.SkinThickness.median(),
    'BloodPressure': diabetes.BloodPressure.median(),
    'Glucose': diabetes.Glucose.median(),
    'BMI': diabetes.BMI.median()
}, inplace=True)

In [31]:
diabetes.SkinThickness.mean()

29.109067017082786

In [34]:
# Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?
# напишем функцию для поиска выбросов
def outliers_iqr(data, feature, log_scale=False, left=1.5, right=1.5):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    Q_25 = x.quantile(0.25)
    Q_75 = x.quantile(0.75)
    IQR = Q_75 - Q_25
    lower_bound = Q_25 - left * IQR
    upper_bound = Q_75 + right * IQR
    outlires = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outlires, cleaned

outlires, cleaned = outliers_iqr(diabetes, 'SkinThickness')
outlires.shape

(87, 8)

In [35]:
# Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?
# функция для поиска выбросов z-методом
def outliers_z_score(data, feature, log_scale=False, left=3, right=3):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outlires, cleaned = outliers_z_score(diabetes, 'SkinThickness')
outlires.shape

(4, 8)

In [37]:
# Найдите число выбросов в DiabetesPedigreeFunction классическим межквартильным рахмахом, а затем в логарифмическом масштабе. 
# Какова разница между двумя этими числами?
outlires, cleaned = outliers_iqr(diabetes, 'DiabetesPedigreeFunction')
print(f'Выбросы классическим методом межквартильного размаха: {outlires.shape}')

outlires, cleaned = outliers_iqr(diabetes, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Выбросы классическим методом межквартильного размаха с логарифмированием: {outlires.shape}')

Выбросы классическим методом межквартильного размаха: (29, 8)
Выбросы классическим методом межквартильного размаха с логарифмированием: (0, 8)
